In [13]:
import os
import re
from bs4 import BeautifulSoup

# removed capitalisation from folder names

al_path = 'data/'
yaokang_path = ''

corpus_path = al_path
files = os.listdir(corpus_path)

In [84]:
cases_souped_dict = {}

for filename in files:
    
    with open(os.path.join(corpus_path, filename), encoding='utf-8') as text:

        soup = BeautifulSoup(text.read(), 'html.parser')

        cases_souped_dict[soup.html.head.title.getText()] = soup.get_text().replace('\n',' ').replace('\t',' ')

    cases_cleaned = {title:text.split('JUDGMENT', 1)[1].split('AustLII:', 1)[0] for title,text in cases_souped_dict.items()}

In [85]:
cases_cleaned

{'AKK15 v Minister for Immigration & Anor [2015] FCCA 2342 (27 August 2015)': '  Applications for the judicial review of decisions of refugee review tribunals must be made within 35 days of the date of the decision sought to be reviewed: s.477(1) of the Migration Act 1958 (Cth).  The court has power to extend the time if an application has been made in writing for that purpose and the Court is satisfied that it is necessary in the interests of the administration of justice to make the order: s.477(2)(b) of the Act. By this application, the applicant seeks an extension of the 35 day time limit so that he might commence an application for judicial review of a decision of a refugee review tribunal that was made on 6 January, 2015.  Any application for review by this court needed to be made by 10 February, 2015.  Using the filing date of the present application as the nominal filing date for his application for review (2 April, 2015), his proposed application for review is eight weeks out 

In [86]:
import string
import collections
 
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint

In [93]:
def process_text(text, stem=True):
    """ Tokenize text and stem words removing punctuation """
    
    # create 'translation table'
    
    transtable = {ord(s):None for s in string.punctuation}
    #transtable[ord('/')] = u' '
    text = text.translate(transtable)
    tokens = word_tokenize(text)
 
    if stem:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(t) for t in tokens]
 
    return tokens

In [89]:
def cluster_texts(texts, clusters=10):
    """ Transform texts to Tf-Idf coordinates and cluster texts using K-Means """
    vectorizer = TfidfVectorizer(tokenizer=process_text,
                                 stop_words=stopwords.words('english'),
                                 max_df=0.9,
                                 min_df=0.2,
                                 lowercase=True)
 
    tfidf_model = vectorizer.fit_transform(texts)
    km_model = KMeans(n_clusters=clusters)
    km_model.fit(tfidf_model)
    
    order_centroids = km_model.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    for i in range(clusters):
        print('Cluster %d:' % i)
    for ind in order_centroids[i, :10]:
        print('%s' % terms[ind])
 
    clustering = collections.defaultdict(list)
 
    for idx, label in enumerate(km_model.labels_):
        clustering[label].append(idx)
 
    return clustering

In [110]:
folder = '/Users/Neuromancer/Documents/neuralcode/2016-04-16-datahack/yaokang/data/'

In [117]:
for name,text in cases_cleaned.items():
    with open(folder+name+'.txt','w') as outfile:
        print(text,file=outfile)

In [124]:
import nltk
my_corpus = nltk.corpus.PlaintextCorpusReader(folder,'.*\.txt') # doctest: +SKIP

In [133]:
my_corpus.words(my_corpus.fileids()[3])

['This', 'is', 'an', 'application', 'for', 'a', ...]